In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import re
import csv

In [2]:
def process_raw_data(city,filename):
    
    data = pd.read_csv("data/" + filename, header=None)
    print("filename", filename)
    print("data", data.head(1))
    print()
    
    for i in data.iterrows():
        resume_id = i[1][0]
        info = i[1][1]

        resume_soup = BeautifulSoup(info, "html.parser")
        containers = resume_soup.findAll(name="div", attrs={"class":"items-container"})

        for container in containers:
            container_type = container['id']

            if container_type == 'work-experience-items':
                all_work = resume_soup.findAll(name="div", attrs={"id":re.compile('workExperience.*')})

                for work_pos in all_work:

                    work_title = work_pos.find(name="p", attrs={"class":"work_title title"})
                    if work_title: work_title = work_title.next
                    else: work_title = ''

                    work_company = work_pos.find(name="div", attrs={"class":"work_company"})
                    if work_company:work_company = work_company.next.next
                    else: work_company = ''

                    addressLocality = work_pos.find(name="span", attrs={"itemprop":"addressLocality"})
                    if addressLocality: addressLocality = addressLocality.next
                    else: addressLocality = ''

                    work_dates = work_pos.find(name="p", attrs={"class":"work_dates"})
                    if work_dates: work_dates = work_dates.next
                    else: work_dates = ''

                    work_descript = work_pos.find(name="p", attrs={"class":"work_description"})
                    if work_descript: work_descript = re.sub('<[^<]+?>', '.', str(work_descript))
                    else: work_descript = ''

                    row = [city, resume_id, container_type, work_title, work_company, addressLocality, work_dates, work_descript]
                    with open('processed_resumes_work.csv', 'a') as myfile:
                        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
                        wr.writerow(row)                

            if container_type == 'education-items':
                all_edu = resume_soup.findAll(name="div", attrs={"class":re.compile("education-section*")})
                for edu in all_edu:
                    edu_title = edu.find(name="p", attrs={"class":"edu_title"})
                    if edu_title: edu_title = edu_title.next
                    else: edu_title = ''

                    edu_school = edu.find(name="div", attrs={"class":"edu_school"})
                    if edu_school: edu_school = edu_school.next.next
                    else: edu_school = ''

                    edu_dates = edu.find(name="p", attrs={"class":"edu_dates"})
                    if edu_dates: edu_dates = edu_dates.next
                    else: edu_dates = ''

                    edu_addressLocality = edu.find(name="span", attrs={"itemprop":"addressLocality"})
                    if edu_addressLocality: edu_addressLocality = edu_addressLocality.next
                    else: edu_addressLocality = ''

                    row = [city, resume_id, container_type, edu_title, edu_school, edu_addressLocality, edu_dates]
                    with open('processed_resumes_education.csv', 'a') as myfile:
                        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
                        wr.writerow(row)                

            if container_type == 'skills-items':
                all_skills = resume_soup.findAll(name="span", attrs={"class":"skill-text"})
                for skill in all_skills:
                    if skill: skill = skill.next
                    else: skill = ''

                    row = [city, resume_id, container_type, skill]
                    with open('processed_resumes_skill.csv', 'a') as myfile:
                        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
                        wr.writerow(row)                

            if container_type == 'additionalinfo-items':
                add_info = resume_soup.find(name="div", attrs={"id":"additionalinfo-section"})
                if add_info: add_info = re.sub('<[^<]+?>', '.', str(add_info))
                else: add_info = ''

                row = [city, resume_id, container_type, add_info]
                with open('processed_resumes_addinfo.csv', 'a') as myfile:
                    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
                    wr.writerow(row)                

            if container_type in ['certification-items','group-items','award-items','patent-items','publication-items','link-items']:
                data_type = container_type.partition('-')[0]
                all_groupings = resume_soup.findAll(name="div", attrs={"class":re.compile(data_type + "-section*")})
                for grouping in all_groupings:

                    title = grouping.find(name="p", attrs={"class": data_type + "_title"})
                    if title: title = title.next
                    else: title = ''

                    date = grouping.find(name="p", attrs={"class": data_type + "_date"})
                    if date: date = date.next
                    else: date = ''

                    url = grouping.find(name="p", attrs={"class": data_type + "_url"})
                    if url: url = url.next.next
                    else: url = ''

                    descript = grouping.find(name="p", attrs={"class": data_type + "_description"})
                    if descript: 
                        descript = re.sub('<[^<]+?>', '.', str(descript))
                    else: descript = ''

                    row = [city, resume_id, container_type, title, date, descript, url]
                    with open('processed_resumes_cert.csv', 'a') as myfile:
                        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
                        wr.writerow(row)

            if container_type == 'military-items':
                all_military = resume_soup.findAll(name="div", attrs={"class":re.compile("military-section*")})
                for military in all_military:

                    military_country = military.find(name="p", attrs={"military_country"})
                    if military_country: 
                        military_country = re.sub('<[^<]+?>', '.', str(military_country))
                    else: military_country = ''

                    military_rank = military.find(name="p", attrs={"military_rank"})
                    if military_rank: 
                        military_rank = re.sub('<[^<]+?>', '.', str(military_rank))
                    else: military_rank = ''

                    military_branch = military.find(name="p", attrs={"military_branch"})
                    if military_branch: 
                        military_branch = re.sub('<[^<]+?>', '.', str(military_branch))
                    else: military_branch = ''

                    military_date = military.find(name="p", attrs={"military_date"})
                    if military_date: military_date = military_date.next
                    else: military_date = ''

                    military_description = military.find(name="p", attrs={"military_description"})
                    if military_description: 
                        military_description = re.sub('<[^<]+?>', '.', str(military_description))
                    else: military_description = ''

                    military_commendations_title = military.find(name="p", attrs={"military_commendations_title"})
                    if military_commendations_title: 
                        military_commendations_title = re.sub('<[^<]+?>', '.', str(military_commendations_title))
                    else: military_commendations_title = ''
                        
                    row = [city, resume_id, container_type, military_country, military_rank, military_branch, \
                           military_date, military_description, military_commendations_title]
                    with open('processed_resumes_military.csv', 'a') as myfile:
                        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
                        wr.writerow(row)

In [3]:
files = [
    'atlanta_resume_data_01.csv',
    'atlanta_resume_data_02.csv',
    'atlanta_resume_data_03.csv',
    'atlanta_resume_data_04.csv',
    'atlanta_resume_data_05.csv',
    'austin_resume_data_01.csv',
    'austin_resume_data_02.csv',
    'austin_resume_data_03.csv',
    'boston_resume_data_01.csv',
    'boston_resume_data_02.csv',
    'boston_resume_data_03.csv',
    'boston_resume_data_04.csv',
    'dc_resume_data_01.csv',
    'dc_resume_data_02.csv',
    'dc_resume_data_03.csv',
    'dc_resume_data_04.csv',
    'denver_resume_data_01.csv',
    'denver_resume_data_02.csv',
    'denver_resume_data_03.csv',
    'denver_resume_data_04.csv',
    'miami_resume_data_01.csv',
    'miami_resume_data_02.csv',
    'miami_resume_data_03.csv',
    'miami_resume_data_04.csv',
    'minn_resume_data_01.csv',
    'minn_resume_data_02.csv',
    'minn_resume_data_03.csv',
    'minn_resume_data_04.csv',
    'minn_resume_data_05.csv',
    'minn_resume_data_06.csv',
    'minn_resume_data_07.csv',
    'nyc_resume_data_01.csv',
    'nyc_resume_data_02.csv',
    'nyc_resume_data_03.csv',
    'nyc_resume_data_04.csv',
    'nyc_resume_data_05.csv',
    'nyc_resume_data_06.csv',
    'nyc_resume_data_07.csv',
    'nyc_resume_data_08.csv',
    'nyc_resume_data_09.csv',
    'seattle_resume_data_01.csv',
    'seattle_resume_data_02.csv',
    'seattle_resume_data_03.csv',
    'sf_resume_data_01.csv',
    'sf_resume_data_02.csv',
    'sf_resume_data_03.csv',
]

In [4]:
for filename in files:
    city = filename.split("_")[0]
    process_raw_data(city,filename)

filename atlanta_resume_data_01.csv
data                   0                                                  1
0  0004d469fc497102  <div class="data_display" id="basic_info_cell"...

filename atlanta_resume_data_02.csv
data                   0                                                  1
0  8b5e3eb44faee609  <div class="data_display" id="basic_info_cell"...

filename atlanta_resume_data_03.csv
data                   0                                                  1
0  a31c3e25400b8075  <div class="data_display" id="basic_info_cell"...

filename atlanta_resume_data_04.csv
data                   0                                                  1
0  b13b69c22d8f7069  <div class="data_display" id="basic_info_cell"...

filename atlanta_resume_data_05.csv
data                   0                                                  1
0  d9042f7aa66a1c51  <div class="data_display" id="basic_info_cell"...

filename austin_resume_data_01.csv
data                   0                     

In [5]:
data = pd.read_csv("processed_resumes_work.csv",header=None)

In [11]:
# data.count()
# data.head(100)
data['resume_id'].nunique()
# data.columns = ['city','resume_id','container','role','company','location','dates','descript']

237037